In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
from collections import defaultdict

In [ ]:
data = pd.read_csv(r"F:\IITB\Leena Ma'am\Summer\INTERACTION-Dataset-TC-v1_0\INTERACTION-Dataset-TC-v1_0\recorded_trackfiles\TC_BGR_Intersection_VA\vehicle_tracks_002.csv")

In [ ]:
no_of_cars = data['track_id'].unique()
car_dict = {car_no: 0.00 for car_no in no_of_cars}

In [ ]:
def generate_autonomous_car_data(timestamp, radius=15, speed=5.398298934, center=np.array([996, 999])):
    # Calculate the angular velocity needed for a circular motion
    angular_velocity = speed / radius  # rad/s
    
    # Calculate the current angle (theta) based on time and angular velocity
    theta = angular_velocity * timestamp  # angle in radians
    
    # Calculate the position on the circle at the current angle
    x = center[0] + radius * np.cos(theta)
    y = center[1] + radius * np.sin(theta)
    
    # Calculate the velocity components (tangential to the circle)
    vx = -speed * np.sin(theta)  # derivative of x with respect to time
    vy = speed * np.cos(theta)   # derivative of y with respect to time
    
    # Calculate the heading angle (psi) as the direction of the velocity vector
    psi_rad = np.arctan2(vy, vx)
    
    return {'x': x, 'y': y, 'vx': vx, 'vy': vy, 'psi_rad': psi_rad}

In [ ]:

grid_size = 30
radius = 15
fov = 120
angle_step = 0.05
total_steps = 500 

In [ ]:
def create_grids(data, grid_size):
    data['grid_x'] = (data['x'] // grid_size).astype(int)
    data['grid_y'] = (data['y'] // grid_size).astype(int)
    return data

grid_radius = int(radius // grid_size)
def get_nearby_grids(autonomous_car, grid_size, radius):
    grid_x = int(autonomous_car['x'] // grid_size)
    grid_y = int(autonomous_car['y'] // grid_size)
    x_range = np.arange(grid_x - grid_radius, grid_x + grid_radius + 1)
    y_range = np.arange(grid_y - grid_radius, grid_y + grid_radius + 1)
    grids = np.array(np.meshgrid(x_range, y_range)).T.reshape(-1, 2)
    #grids = [(i, j) for i in range(grid_x - grid_radius, grid_x + grid_radius + 1)
    #                 for j in range(grid_y - grid_radius + 1)]
    return grids

In [ ]:
def get_car_boundaries(car):
 
    center_x, center_y = car['x'], car['y']
    half_length = car['length']/2
    half_width = car['width']/2
    angle = np.arctan2(car['vy'], car['vx'])

    corners = np.array([
        [center_x - half_length * np.cos(angle) - half_width * np.sin(angle), 
         center_y - half_length * np.sin(angle) + half_width * np.cos(angle)],  # Top-left
        [center_x + half_length * np.cos(angle) - half_width * np.sin(angle), 
         center_y + half_length * np.sin(angle) + half_width * np.cos(angle)],  # Top-right
        [center_x + half_length * np.cos(angle) + half_width * np.sin(angle), 
         center_y + half_length * np.sin(angle) - half_width * np.cos(angle)],  # Bottom-right
        [center_x - half_length * np.cos(angle) + half_width * np.sin(angle), 
         center_y - half_length * np.sin(angle) - half_width * np.cos(angle)]   # Bottom-left
    ])


    return corners

In [ ]:
def filter_cars_by_grid_and_fov(data, grids, autonomous_car, radius, fov, current_time):
    fov_half_angle = np.radians(fov) / 2
    data['relevant'] = False  # Whether the car is relevant (inside radius and FOV)
    
    for _, car in data.iterrows():
        car_id = car['track_id']
        corners = get_car_boundaries(car)  # Get car's boundary corners

        # Compute distances of all corners from the autonomous car
        distances = np.sqrt((corners[:, 0] - autonomous_car['x'])**2 + (corners[:, 1] - autonomous_car['y'])**2)

        # Compute car grids
        car_grids = [(int(x // grid_size), int(y // grid_size)) for x, y in corners]
        car_grids = list(set(car_grids))  # Remove duplicate grids

        # Check if car's grids overlap with nearby grids
        if any(grid in grids for grid in car_grids):
            # Check if the car is within the radius
            if np.any(distances <= radius):
                # Check if the car is within the FOV
                relative_angles = np.arctan2(corners[:, 1] - autonomous_car['y'], corners[:, 0] - autonomous_car['x'])
                car_angles = np.mod(relative_angles - autonomous_car['psi_rad'] + np.pi, 2 * np.pi) - np.pi

                if np.any(np.abs(car_angles) <= fov_half_angle):
                    data.at[car.name, 'relevant'] = True
                    continue  # Skip updating `car_dict` for relevant cars
            
        # **Fix: Ensure `distances` is always available**
        velocity = np.linalg.norm([car['vx'], car['vy']])  # Calculate speed
        if velocity > 0:
            min_time_to_reach = max(0, (np.min(distances) - radius) / velocity)
        else:
            min_time_to_reach = float('inf')  # Car is stationary

        # Update car_dict with the next eligible time
        car_dict[car_id] = current_time + min_time_to_reach

    # Return only relevant cars
    return data[data['relevant']]



# Discretize angles for efficient angle-based filtering
fov_half_angle = np.radians(fov) / 2
discrete_angles = np.arange(-fov_half_angle, fov_half_angle + np.radians(angle_step), np.radians(angle_step))
def discretize_angles(data, autonomous_car, angle_step):

    data_copy = data.copy()
    data_copy['discrete_angle'] = None
    
    for _, car in data_copy.iterrows():
        corners = get_car_boundaries(car)
        relative_angles = np.arctan2(corners[:, 1] - autonomous_car['y'], corners[:, 0] - autonomous_car['x'])
        car_angles = (relative_angles - autonomous_car['psi_rad'] + np.pi)% (2 * np.pi) - np.pi

        if np.any(np.abs(car_angles) <= fov_half_angle):
            data_copy.at[car.name, 'discrete_angle'] = np.digitize(car_angles[np.abs(car_angles) <= fov_half_angle], discrete_angles) * angle_step

    return data_copy

In [ ]:
angle_mult = 180 / np.pi
def get_angle(car_center, point):
    
    return np.arctan2(point[1] - car_center[1], point[0] - car_center[0]) * angle_mult

def is_perpendicular(car, autonomous_car):
    
    car_center = car['x'], car['y']
    center = autonomous_car['x'], autonomous_car['y']
    vector_to_center = np.array(car_center) - np.array(center)
    velocity_vector = np.array([car['vx'], car['vy']])
    dot_product = np.dot(vector_to_center, velocity_vector)
    return np.isclose(dot_product, 0, atol=1e-3)


def is_parallel(car, autonomous_car):
    
    center = autonomous_car['x'], autonomous_car['y']
    car_center = car['x'], car['y']
    vector_to_center = np.array(car_center) - np.array(center)
    vector_to_center = np.append(vector_to_center, 0)
    velocity_vector = np.array([car['vx'], car['vy'], 0])
    cross_product = np.cross(vector_to_center, velocity_vector)
    return np.isclose(cross_product[2], 0, atol=1e-3)

def line_intersection(p1, p2, q1, q2):

    # Line p1 -> p2: (x1, y1) to (x2, y2)
    # Line q1 -> q2: (x3, y3) to (x4, y4)
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = q1
    x4, y4 = q2
    
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    
    if np.isclose(denom, 0):  # Lines are parallel or coincident
        return None
    
    t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denom
    u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denom
    
    # Check if intersection is within the segments
    if 0 <= t <= 1 and 0 <= u <= 1:
        intersection_x = x1 + t * (x2 - x1)
        intersection_y = y1 + t * (y2 - y1)
        return np.array([intersection_x, intersection_y])
    else:
        return None


def check_visibility(autonomous_car, other_cars, fov_angle, radius):
    visible_segments = []
    #print(f'type of other_cars is {type(other_cars)}')
    # Define FOV rays extending from the autonomous car
    fov_left_angle = autonomous_car['psi_rad'] - np.radians(fov_angle) / 2
    fov_right_angle = autonomous_car['psi_rad'] + np.radians(fov_angle) / 2
    center = autonomous_car['x'], autonomous_car['y']
    fov_left_ray = np.array([
        center,
        center + radius * np.array([np.cos(fov_left_angle), np.sin(fov_left_angle)])
    ])
    
    fov_right_ray = np.array([
        center,
        center + radius * np.array([np.cos(fov_right_angle), np.sin(fov_right_angle)])
    ])

    if len(other_cars) != 0:
        for _, car in other_cars.iterrows():


            corners = get_car_boundaries(car)
            car_x_min = min(corners[:, 0])
            car_x_max = max(corners[:, 0])
            car_y_min = min(corners[:, 1])
            car_y_max = max(corners[:, 1])
            
            if car_x_min <= center[0] <= car_x_max and car_y_min <= center[1] <= car_y_max:
                # Skip this car if the autonomous car is inside it
                continue

            visible_corners = []
            corner_distances = []

            # Find corners inside FOV
            for corner in corners:
                corner_angle = get_angle(center, corner)
                relative_angle = corner_angle - np.degrees(autonomous_car['psi_rad'])
                relative_angle = (relative_angle + 180) % 360 - 180  # Normalize angle
                
                if abs(relative_angle) <= fov_angle / 2:
                    visible_corners.append(corner)
                    corner_distances.append(np.linalg.norm(corner - center))

            nearest_left_intersection = None
            nearest_right_intersection = None
            min_left_distance = float('inf')
            min_right_distance = float('inf')

            if len(visible_corners) > 1:
                distances = np.linalg.norm(np.array(visible_corners) - np.array(center), axis=1)
                farthest_corner_idx = np.argmax(distances)
                # visible_corners.pop(farthest_corner_idx)
                if len(visible_corners) > 2:
                    visible_corners.pop(farthest_corner_idx)                
                else:
                    # Case when len(visible_corners) == 2
                    for corner in corners:
                        print(f'Get angle o/p is {get_angle(center,corner)}')
                    print(f'psi_rad is {180*autonomous_car["psi_rad"]/np.pi}')
                    angles = [get_angle(center, corner) - 180*(autonomous_car['psi_rad'])/np.pi for corner in visible_corners]
                    angles = [(angle + 180) % 360 - 180 for angle in angles]  # Normalize to [-180, 180]

                    nearest_corner_idx = np.argmin(distances)  # Nearest corner index
                    farthest_corner_idx = 1 - nearest_corner_idx  # Other corner index
                    print(f'farthest corner angle is {angles[farthest_corner_idx]}')
                    print(f'nearest corner angle is {angles[nearest_corner_idx]}')
                    # Check if the second corner is closer to the FOV boundary (-60° or 60°)
                    ax, ay, vx, vy = autonomous_car['x'], autonomous_car['y'], autonomous_car['vx'], autonomous_car['vy']
                    
                    signs = []
                    
                    for corner in corners:
                        cx, cy = corner
                        vec_x, vec_y = cx - ax, cy - ay  # Vector from car to corner
                        cross_product = vx * vec_y - vy * vec_x  # 2D cross product
                        signs.append(np.sign(cross_product))  # Store sign (+1, -1, or 0)
                    print(f'signs are {signs}')
                    # if all(s == signs[0] for s in signs):
                    #     if abs(angles[farthest_corner_idx]) <= abs(angles[nearest_corner_idx]):  # Adjustable threshold (was 60, made it softer)
                    #         visible_corners.pop(farthest_corner_idx)              
                    if angles[farthest_corner_idx]*angles[nearest_corner_idx] > 0:
                        if all(s == signs[0] for s in signs):
                            if abs(angles[farthest_corner_idx]) >= abs(angles[nearest_corner_idx]):  # Adjustable threshold (was 60, made it softer)
                                visible_corners.pop(farthest_corner_idx)  # Remove the farthest corner
                        else:
                            if angles[farthest_corner_idx] >= angles[nearest_corner_idx]:
                                visible_corners.pop(farthest_corner_idx)
                    else:
                        visible_corners.pop(farthest_corner_idx)

            if len(visible_corners) < 4:
                all_intersections = []
                for i in range(4):
                    p1, p2 = corners[i], corners[(i + 1) % 4]

                    # Check for intersection with FOV left ray
                    left_intersection = line_intersection(p1, p2, fov_left_ray[0], fov_left_ray[1])
                    if left_intersection is not None:
                        left_dist = np.linalg.norm(left_intersection - center)
                        if left_dist < min_left_distance:
                            min_left_distance = left_dist
                            nearest_left_intersection = left_intersection
                        all_intersections.append(left_intersection)

                    # Check for intersection with FOV right ray
                    right_intersection = line_intersection(p1, p2, fov_right_ray[0], fov_right_ray[1])
                    if right_intersection is not None:
                        right_dist = np.linalg.norm(right_intersection - center)
                        if right_dist < min_right_distance:
                            min_right_distance = right_dist
                            nearest_right_intersection = right_intersection
                        all_intersections.append(right_intersection)

                if len(visible_corners) == 1:
                    
                    if nearest_left_intersection is not None and nearest_right_intersection is not None:
                        visible_segments.append((nearest_left_intersection, nearest_right_intersection))
                    elif nearest_left_intersection is not None:
                        visible_segments.append((visible_corners[0], nearest_left_intersection))
                    elif nearest_right_intersection is not None:
                        visible_segments.append((visible_corners[0], nearest_right_intersection))
                
                else:
                    
                    if nearest_left_intersection is not None:
                        visible_corners.append(nearest_left_intersection)
                    if nearest_right_intersection is not None:
                        visible_corners.append(nearest_right_intersection)

            # Find the nearest corner to autonomous car
            if visible_corners:
                corner_distances = np.linalg.norm(np.array(visible_corners) - center, axis=1)
                nearest_corner_idx = np.argmin(corner_distances)
                nearest_corner = visible_corners[nearest_corner_idx]

                # Determine visible segment
                if is_perpendicular(car, autonomous_car):
                    car_vel = np.arctan2(car['vy'], car['vx'])
                    i = 1 if abs(get_angle(nearest_corner, visible_corners[(nearest_corner_idx + 1) % len(visible_corners)]) - (car_vel * 180 / np.pi)) < 1e-3 else -1
                    visible_segments.append((nearest_corner, visible_corners[(nearest_corner_idx + i) % len(visible_corners)]))
                elif is_parallel(car, autonomous_car):
                    car_vel = car['vx'], car['vy']
                    i = 1 if abs(np.dot(nearest_corner - visible_corners[(nearest_corner_idx + 1) % len(visible_corners)], car_vel)) < 1e-3 else -1
                    visible_segments.append((nearest_corner, visible_corners[(nearest_corner_idx + i) % len(visible_corners)]))
                else:
                    # Shade two sides visible
                    next_corner_1 = visible_corners[(nearest_corner_idx + 1) % len(visible_corners)]
                    next_corner_2 = visible_corners[(nearest_corner_idx - 1) % len(visible_corners)]
                    visible_segments.append((nearest_corner, next_corner_1))
                    visible_segments.append((nearest_corner, next_corner_2))

    return visible_segments


In [ ]:
# Initialize data
data = create_grids(data, grid_size)

# Plot setup
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_title('Relevant and Non-Relevant Cars within FOV')

unique_timestamps = data['timestamp_ms'].unique()[:100]

# Function to plot each car as a rectangle
def plot_car(ax, car, is_relevant):
    corners = get_car_boundaries(car)
    polygon = plt.Polygon(corners, closed=True, edgecolor='red' if is_relevant else 'blue', facecolor='none')
    ax.add_patch(polygon)

# Update function for animation
#fov_lines = [ax.plot([], [], 'b--')[0] for _ in range(2)]
#autonomous_car_scatter = ax.scatter([], [], c='blue', label='Autonomous Car', s=10)
def update(frame):
    start_frame = time.time()
    step = frame
    current_time = unique_timestamps[frame]
    timestamp_data = data[data['timestamp_ms'] == current_time]
    filtered_data = []
    for _, car in timestamp_data.iterrows():
        car_id = car['track_id']
        if car_id in car_dict:
            if current_time < car_dict[car_id]:
                continue  # Skip cars not eligible for processing
        filtered_data.append(car)
    timestamp_data = pd.DataFrame(filtered_data)
    
    autonomous_car = generate_autonomous_car_data(step)
    nearby_grids = get_nearby_grids(autonomous_car, grid_size, radius)
    nearby_cars = filter_cars_by_grid_and_fov(timestamp_data, nearby_grids, autonomous_car, radius, fov, current_time)
    relevant_cars = discretize_angles(nearby_cars, autonomous_car, angle_step)
    # print(f'type of relevant_Cars is {type(relevant_cars)} and number of relevant cars = {len(relevant_cars)}')
    rel_cars_no.append(len(relevant_cars))
    ax.clear()
    
    # Plot autonomous car
    #autonomous_car_scatter.set_offsets([[autonomous_car['x'], autonomous_car['y']]])
    ax.scatter(autonomous_car['x'], autonomous_car['y'], c='blue', label='Autonomous Car', s=10)
    ax.set_title(step)
    # Draw FOV lines
    angle = autonomous_car['psi_rad']
    x, y = autonomous_car['x'], autonomous_car['y']
    fov_left = angle - np.radians(fov) / 2
    fov_right = angle + np.radians(fov) / 2

    #fov_lines[0].set_data([x, x + radius * np.cos(fov_left)], [y, y + radius * np.sin(fov_left)])
    #fov_lines[1].set_data([x, x + radius * np.cos(fov_right)], [y, y + radius * np.sin(fov_right)])
    ax.plot([x, x + radius * np.cos(fov_left)], [y, y + radius * np.sin(fov_left)], 'b--')
    ax.plot([x, x + radius * np.cos(fov_right)], [y, y + radius * np.sin(fov_right)], 'b--')
    
    ax.set_xlim(971,1021)
    ax.set_ylim(980, 1018)
    
    visible_segments = check_visibility(autonomous_car, relevant_cars, fov, radius)
    # visible_points = check_visibility(autonomous_car, relevant_cars, fov, radius)
    # ax.scatter(visible_points[:, 0], visible_points[:, 1], color='g', s=2)  # LiDAR-like points

    # Plot relevant cars
    for _, car in relevant_cars.iterrows():
        plot_car(ax, car, is_relevant=True)

    for seg in visible_segments:
        ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'g-', linewidth=1)


    # Plot non-relevant cars
    if 'track_id' in nearby_cars.columns:
        non_relevant_cars = nearby_cars[~nearby_cars['track_id'].isin(relevant_cars['track_id'])]
        for _, car in non_relevant_cars.iterrows():
            plot_car(ax, car, is_relevant=False)

    # plt.legend()
    plt.draw()
    # for _, car in timestamp_data.iterrows():
    #     car_id = car['track_id']
    #     position = np.array([car['x'], car['y']])
    #     velocity = np.linalg.norm([car['vx'], car['vy']])  # Calculate speed from velocity components
    #     distance = np.linalg.norm(position - np.array([autonomous_car['x'], autonomous_car['y']]))
    #     if distance < radius:
    #         continue  # Car is already within the radius
    #     if velocity > 0:
    #         min_time_to_reach = max(0, (distance - radius) / velocity)
    #     else:
    #         min_time_to_reach = float('inf')  # Car isn't moving, so it won't reach the radius

    #     car_dict[car_id] = current_time + min_time_to_reach
    end_frame = time.time()
    # print(f"Time taken to execute frame {frame} is {end_frame-start_frame}")
    times.append(end_frame-start_frame)




In [ ]:
times = []
rel_cars_no =[]
# for frame in range(len(unique_timestamps)):
#     update(frame)
ani = animation.FuncAnimation(fig, update, frames=len(unique_timestamps), interval=1000, repeat=False)
ani.save(r"F:\IITB\Leena Ma'am\Summer\Working versions\2D_cars_line_blue_red_testing.gif", writer='pillow')

fig1, ax1 = plt.subplots()

# Plotting the first dataset
ax1.plot(times, color="blue")
ax1.set_xlabel("X-axis")
ax1.set_ylabel("Y1-axis (left)", color="blue")
ax1.tick_params(axis="y", labelcolor="blue")
ax1.set_ylim(0,0.40)

# Creating a second y-axis
ax2 = ax1.twinx()
ax2.plot(rel_cars_no, color="red")
ax2.set_ylabel("Y2-axis (right)", color="red")
ax2.tick_params(axis="y", labelcolor="red")
# plt.savefig(f"Plots/trial23031850.png")
plt.show()
# print(times)
print(f"Frame Rate Output of SPART is {len(times)/sum(times)}")
# print(f"Average cars per times stamp is {sum(rel_cars_no)/len(rel_cars_no)}")